In [2]:
!pip install transformers trl accelerate torch bitsandbytes peft datasets -qU

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.0 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.2 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which

In [4]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, GenerationConfig, LlamaForCausalLM, LlamaTokenizer
from sklearn.model_selection import train_test_split
import transformers
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import copy
#import loralib as lora
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
#from peft import LoraConfig
from trl import SFTTrainer

In [5]:
import os

In [ ]:
#

In [23]:
import pandas as pd

# Specify the path to your Excel file
PATH = "/kaggle/input/465analysisis/Our_trainset.xlsx"

try:
    # Read the Excel file into a DataFrame
    trainset= pd.read_excel(PATH)

    # Now you can work with the 'dataset' DataFrame

    # For example, you can print the first few rows of the DataFrame
    print(trainset.head())

except Exception as e:
    print(f"Error loading dataset: {e}")


                                          Main_Paper  \
0  Background  The available evidence about the p...   
1  Introduction Several studies have demonstrated...   
2  Background  Glycemic control is vital to patie...   
3  Objective  This meta-analysis aimed to evaluat...   
4  Background  Recurrent hemarthrosis is one of t...   

                                                  MP  
0  MP: ObjectiveSeveral randomized controlled tri...  
1  MP:Background: Depression is a severe disease ...  
2  MP:Purpose  Diabetes mellitus (DM) is increasi...  
3  MP: Background Recent in vitro and animal expe...  
4  MP:Background: The primary clinical manifestat...  


In [24]:
import pandas as pd

# Specify the path to your Excel file
PATH = "/kaggle/input/465analysisis/Our_testset.xlsx"

try:
    # Read the Excel file into a DataFrame
    testset = pd.read_excel(PATH)

    # Now you can work with the 'dataset' DataFrame

    # For example, you can print the first few rows of the DataFrame
    print(testset.head())

except Exception as e:
    print(f"Error loading dataset: {e}")


                                          Main_Paper  \
0  Mindfulness is characterized by skills such as...   
1  Teen Psychological Dating Violence (TPDV) is a...   
2  Objective This study aims to quantitatively ev...   
3  We investigated whether the addition of adjuva...   
4  Objectives This study investigated the prevale...   

                                                  MP  
0  MP: Posttraumatic stress disorder (PTSD) is a ...  
1  MP: The Safe Dates Project is a randomized tri...  
2  MP: Background: Intermittent theta burst stimu...  
3  MP: The present study aimed to retrospectively...  
4  MP: Campylobacter spp. are among the leading f...  


In [25]:

# Specify the path to your Excel file
PATH = "/kaggle/input/465analysisis/Our_validationset.xlsx"

try:
    # Read the Excel file into a DataFrame
    validationset = pd.read_excel(PATH)

    # Now you can work with the 'dataset' DataFrame

    # For example, you can print the first few rows of the DataFrame
    print(validationset.head())

except Exception as e:
    print(f"Error loading dataset: {e}")


                                          Main_Paper  \
0  Acute leukemia commonly occurs in young childr...   
1  Nonalcoholic fatty liver disease (NAFLD) is a ...   
2  Neurodegenerative diseases (NDs) have posed si...   
3  The associations between vegetable intake and ...   
4  The use of nutraceutical products to enhance m...   

                                                  MP  
0  MP:Breastfeeding has a protective impact upon ...  
1  MP: Selpercatinib, a highly selective potent a...  
2  MP:The aim of the present study is to evaluate...  
3  MP:Higher levels of vegetable consumption have...  
4  MP:Infertility is the condition of about 15% o...  


In [26]:
dataset

,Main_Paper,MP
0,Background The available evidence about the p...,MP: ObjectiveSeveral randomized controlled tri...
1,Introduction Several studies have demonstrated...,MP:Background: Depression is a severe disease ...
2,Background Glycemic control is vital to patie...,MP:Purpose Diabetes mellitus (DM) is increasi...
3,Objective This meta-analysis aimed to evaluat...,MP: Background Recent in vitro and animal expe...
4,Background Recurrent hemarthrosis is one of t...,MP:Background: The primary clinical manifestat...
5,"Background Curcumin, a natural polyphenolic c...",MP:Polycystic ovary syndrome (PCOS) is a heter...
6,Purpose Despite the high prevalence of postdi...,MP:Background The aetiology of postdialysis f...
7,Purpose Research has shown that prone positio...,MP:OBJECTIVES: To determine the characteristic...
8,"Handhelds (e.g., cell phones, tablets) are pro...",MP:The importance of mobile technologies in th...
9,Background Circulating concentrations of homo...,MP:Objectives To investigate the associations ...


In [27]:
testset

,Main_Paper,MP
0,Mindfulness is characterized by skills such as...,MP: Posttraumatic stress disorder (PTSD) is a ...
1,Teen Psychological Dating Violence (TPDV) is a...,MP: The Safe Dates Project is a randomized tri...
2,Objective This study aims to quantitatively ev...,MP: Background: Intermittent theta burst stimu...
3,We investigated whether the addition of adjuva...,MP: The present study aimed to retrospectively...
4,Objectives This study investigated the prevale...,MP: Campylobacter spp. are among the leading f...
5,Background Previous reviews reported that the ...,MP: Ubiquinone is a carrier of the mitochondri...
6,Objective To explore the protective effect and...,MP: Ethnopharmacological relevance Danshen Inj...
7,Objective To explore the protective effect and...,MP: This study examined the neurobiology of ma...
8,Event-related potentials (ERPs) are an excelle...,MP: Parental reflective functioning refers to ...
9,Background Direct oral anticoagulants (DOACs) ...,MP: Background The novel oral anticoagulants (...


In [11]:
#valset

NameError: name 'valset' is not defined

In [28]:
dataset=dataset.head(10)

In [29]:
trainset

,Main_Paper,MP
0,Background The available evidence about the p...,MP: ObjectiveSeveral randomized controlled tri...
1,Introduction Several studies have demonstrated...,MP:Background: Depression is a severe disease ...
2,Background Glycemic control is vital to patie...,MP:Purpose Diabetes mellitus (DM) is increasi...
3,Objective This meta-analysis aimed to evaluat...,MP: Background Recent in vitro and animal expe...
4,Background Recurrent hemarthrosis is one of t...,MP:Background: The primary clinical manifestat...
...,...,...
395,"Despite difficulties in reading and writing, s...",MP:This study aimed to verify if the creativit...
396,The relationship between smoking and the devel...,MP:Data on the association between smoking and...
397,To use meta-analytic techniques to examine the...,"MP:A prospective, randomized, double-blind stu..."
398,Available on the association between the Medi...,MP:The aim of this study is to evaluate the as...


In [ ]:
# from sklearn.model_selection import train_test_split
# import datasets
# # dataset['tokenized_label'] = llama_tokenizer(dataset['Label'], return_tensors='pt')

# trainset, testset = train_test_split(
#     dataset,
#     test_size=0.2
# )
# trainset = datasets.Dataset.from_dict(trainset)
# testset = datasets.Dataset.from_dict(testset)

# print(len(trainset))
# print(len(testset))

In [15]:
!pip install langchain 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.2/806.2 kB 21.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [30]:
import pandas as pd
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Instantiate RecursiveCharacterTextSplitter with custom parameters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    length_function=len,
    #separators=['\n\n','MP:', '\n', ' ', '']
)

# Function to split text into chunks using the text splitter
def split_text_into_chunks(text):
    return text_splitter.split_text(text)

In [32]:
train_df = pd.DataFrame({"Paper": trainset["Main_Paper"], "Meta_Paper": trainset["MP"]})
chunked_train_data = {"Paper": [], "Meta_Paper": []}

# Iterate through each row in the dataset
for index, row in train_df.iterrows():  # Change trainset to train_df
    context = row["Meta_Paper"]
    label = row["Paper"]

    # Split the context into overlapping chunks
    chunks = split_text_into_chunks(context)
    for chunk in chunks:
        chunked_train_data["Paper"].append(label)
        chunked_train_data["Meta_Paper"].append(chunk)

# Create a new DataFrame with the chunked data
chunked_train_df = pd.DataFrame(chunked_train_data)


In [33]:
trainset.isnull()

,Main_Paper,MP
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
395,False,False
396,False,False
397,False,False
398,False,False


In [34]:
train_df = pd.DataFrame({"Paper": trainset["Main_Paper"], "Meta_Paper": trainset["MP"]})

In [35]:
train_df

,Paper,Meta_Paper
0,Background The available evidence about the p...,MP: ObjectiveSeveral randomized controlled tri...
1,Introduction Several studies have demonstrated...,MP:Background: Depression is a severe disease ...
2,Background Glycemic control is vital to patie...,MP:Purpose Diabetes mellitus (DM) is increasi...
3,Objective This meta-analysis aimed to evaluat...,MP: Background Recent in vitro and animal expe...
4,Background Recurrent hemarthrosis is one of t...,MP:Background: The primary clinical manifestat...
...,...,...
395,"Despite difficulties in reading and writing, s...",MP:This study aimed to verify if the creativit...
396,The relationship between smoking and the devel...,MP:Data on the association between smoking and...
397,To use meta-analytic techniques to examine the...,"MP:A prospective, randomized, double-blind stu..."
398,Available on the association between the Medi...,MP:The aim of this study is to evaluate the as...


In [36]:


# Define chunk size and overlapping amount
# chunk_size = 1500  # Adjust this as needed
# overlap = 500  # Adjust this as needed

# Create a list to store chunked training data
chunked_train_data = {"Paper": [], "Meta_Paper": []}

for i, row in train_df.iterrows():
    context = row["Meta_Paper"]
    label = row["Paper"]
    #print(len(context))
    
    # Split the context into overlapping chunks
    chunks = split_text_into_chunks(context)
    for chunk in chunks:
        chunked_train_data["Paper"].append(label)
        chunked_train_data["Meta_Paper"].append(chunk)

# Create a new DataFrame with the chunked data
chunked_train_df = pd.DataFrame(chunked_train_data)



In [37]:
chunked_train_df

,Paper,Meta_Paper
0,Background The available evidence about the p...,MP: ObjectiveSeveral randomized controlled tri...
1,Background The available evidence about the p...,"hormonal and inflammatory parameters, or can i..."
2,Background The available evidence about the p...,and synbiotics supplementation in lipid profil...
3,Background The available evidence about the p...,and meta-analysis. Mean Difference (MD) was po...
4,Background The available evidence about the p...,on homeostatic model assessment-insulin resist...
...,...,...
3835,Available on the association between the Medi...,"incidents, histologically confirmed gastric ca..."
3836,The effective management of the 33 million chi...,MP: Despite the decreasing rate of under nutri...
3837,The effective management of the 33 million chi...,No adverse reactions were observed. There were...
3838,The effective management of the 33 million chi...,Three-factor analysis of covariance of the eff...


In [39]:
val_df = pd.DataFrame({"Paper": validationset["Main_Paper"], "Meta_Paper": validationset["MP"]})

# Define chunk size and overlapping amount
# chunk_size = 1500  # Adjust this as needed
# overlap = 500  # Adjust this as needed

# Create a list to store chunked training data
chunked_val_data = {"Paper": [], "Meta_Paper": []}

for i, row in val_df.iterrows():
    context = row["Meta_Paper"]
    label = row["Paper"]

    # Split the context into overlapping chunks
    chunks = split_text_into_chunks(context)
    for chunk in chunks:
        chunked_val_data["Paper"].append(label)
        chunked_val_data["Meta_Paper"].append(chunk)

# Create a new DataFrame with the chunked data
chunked_val_df = pd.DataFrame(chunked_val_data)

In [40]:
test_df = pd.DataFrame({"Paper": testset["Main_Paper"], "Meta_Paper": testset["MP"]})

# Define chunk size and overlapping amount
# chunk_size = 1500  # Adjust this as needed
# overlap = 500  # Adjust this as needed

# Create a list to store chunked training data
chunked_test_data = {"Paper": [], "Meta_Paper": []}

for i, row in test_df.iterrows():
    context = row["Meta_Paper"]
    label = row["Paper"]

    # Split the context into overlapping chunks
    chunks = split_text_into_chunks(context)
    for chunk in chunks:
        chunked_test_data["Paper"].append(label)
        chunked_test_data["Meta_Paper"].append(chunk)

# Create a new DataFrame with the chunked data
chunked_test_df = pd.DataFrame(chunked_test_data)

In [41]:
import datasets

In [42]:
trainset = datasets.Dataset.from_dict(chunked_train_df)
valset = datasets.Dataset.from_dict(chunked_val_df)
testset = datasets.Dataset.from_dict(chunked_test_df)

In [21]:
#from huggingface_hub import notebook_login

In [22]:
#notebook_login()

In [43]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    device_map='auto',
    quantization_config=nf4_config,
    use_cache=False
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [44]:
DEFAULT_SYSTEM_PROMPT = """
Given a collection of abstracts from papers used in various medical fields, generate a meta-analysis abstract summarizing the key findings of those abstracts and provide numerical values or statistical information for specific observations that are commonly reported in the provided abstracts. Some provided abstracts may have chunks, so maintain information similarities.
""".strip()

In [45]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['Meta_Paper'])):
        text = f"### {DEFAULT_SYSTEM_PROMPT},### Abstracts: {example['Meta_Paper'][i]}  \n ### Meta-Analysis Abstract: {example['Paper'][i]}"
        

        output_texts.append(text)
    return output_texts



In [47]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

In [49]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [28]:
'''wandb_project = "llama2_465finetune22"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project'''

In [50]:
import torch.nn as nn
import torch.nn.functional as F

In [51]:
def softmax_selection(predictions, temperature=1.0, dim=-1):
    """
    Apply softmax to model predictions and sample a token based on the resulting probabilities.

    Args:
        predictions (torch.Tensor): The tensor containing the raw predictions from the model.
        temperature (float): Temperature parameter to adjust the sharpness of the probability distribution.
                              A lower temperature makes the distribution sharper.

    Returns:
        torch.Tensor: Tensor containing the selected token IDs.
    """
   # print(predictions.dim())
    if predictions.dim() > 2:
        predictions = predictions.view(-1, predictions.size(-1))
    # Apply softmax with temperature
    probs = F.softmax(predictions / temperature, dim=-1)
   # print(probs.dim())
#     if probs.dim()>2:
#         probs.squeeze()
        
#    print(probs.dim())
    

     #Reshape probs tensor if necessary
  

    # Sampling a token based on the probabilities
    sampled_tokens = torch.multinomial(probs, 1)

    return sampled_tokens

In [52]:
def quantities(decoded_predictions, decoded_labels):
    """
    Extract quantities from decoded predictions and labels.

    Args:
        decoded_predictions (list): List of decoded predictions.
        decoded_labels (list): List of decoded labels.

    Returns:
        tuple: A tuple containing lists of predicted quantities and actual quantities.
    """
    predicted_quantities = []
    actual_quantities = []

    for prediction, label in zip(decoded_predictions, decoded_labels):
        # Placeholder: Extracting the first number encountered in the decoded strings
        predicted_quantity = extract_quantity(prediction)
        actual_quantity = extract_quantity(label)
        predicted_quantities.append(predicted_quantity)
        actual_quantities.append(actual_quantity)

    return predicted_quantities, actual_quantities

def extract_quantity(text):
    """
    Extract the first numeric value from a string.

    Args:
        text (str): Input string.

    Returns:
        float: Extracted numeric value or NaN if no numeric value found.
    """
    import re
    # Regular expression to match numeric values
    numeric_pattern = re.compile(r'[-+]?[0-9]*\.?[0-9]+')
    # Search for numeric values in the text
    match = re.search(numeric_pattern, text)
    if match:
        # Extract and convert the first numeric value found
        return float(match.group())
    else:
        # Return NaN if no numeric value found
        return float('nan')

In [53]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="./mistral_instruct_generation",
    num_train_epochs=10,  # Uncomment this line to train for 10 epochs
    per_device_train_batch_size=4,
    warmup_steps=0.03,
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    eval_steps=20,
    learning_rate=2e-4,
    fp16=True,
    lr_scheduler_type='constant',
) 

In [57]:
trainset

Dataset({
    features: ['Paper', 'Meta_Paper'],
    num_rows: 3840
})

In [58]:
testset

Dataset({
    features: ['Paper', 'Meta_Paper'],
    num_rows: 512
})

In [59]:
from sklearn.model_selection import train_test_split
import datasets
#dataset['tokenized_label'] = llama_tokenizer(dataset['Label'], return_tensors='pt')
trainset, testset = train_test_split(
    dataset,
    test_size=0.2
)
trainset = datasets.Dataset.from_dict(trainset)
testset = datasets.Dataset.from_dict(testset)
type(trainset)

datasets.arrow_dataset.Dataset

In [61]:
trainset

Dataset({
    features: ['Main_Paper', 'MP'],
    num_rows: 8
})

In [ ]:
from trl import SFTTrainer

max_seq_length = 1000

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=formatting_prompts_func,
  args=args,
  train_dataset=trainset,  # Use your training dataset here
  eval_dataset=testset     # Use your testing dataset here
)

In [30]:


class CustomSFTTrainer(SFTTrainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def compute_loss(self, model, inputs, return_outputs=False):
        # Extract and remove labels from inputs
        labels = inputs.get("labels")
        
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits  # Assuming the model outputs logits
        
        # Ensure logits and labels are float tensors
        logits = logits.float()
        labels = labels.float()
        
        # Reshape logits and labels
        logits = logits.view(logits.size(0), logits.size(1), -1)  
        labels = labels.view(labels.size(0), labels.size(1), -1)  
        
        # Normalize embeddings for cosine similarity
        logits = F.normalize(logits, p=2, dim=-1)
        labels = F.normalize(labels, p=2, dim=-1)

        # Calculate cosine similarity
        cosine_sim = torch.cosine_similarity(logits, labels, dim=-1)
        
        # Calculate loss using inverse of cosine similarity with epsilon for numerical stability
        epsilon = torch.tensor(1e-8, device=logits.device)
        loss = torch.mean(1 / (cosine_sim + epsilon))

        return (loss, outputs) if return_outputs else loss


In [32]:

# Training Params
train_params = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    fp16=False,
    max_grad_norm=0.3,
    num_train_epochs=2,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    save_safetensors=True,
    lr_scheduler_type="constant",
    output_dir="./results_modified_test",
    #num_train_epochs=1,
    #per_device_train_batch_size=2,
    #gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=2,
    logging_steps=2,
    learning_rate=2e-4,
    weight_decay=0.001,
    #fp16=False,
    bf16=False,
    #max_grad_norm=0.3,
    max_steps=-1,
    #warmup_ratio=0.05,
    #group_by_length=True,
    #lr_scheduler_type="constant",
    report_to="wandb",   
    # push to hub parameters,
   # push_to_hub=True,
)

# Trainer
fine_tuning = CustomSFTTrainer(
    model=base_model,
    train_dataset=trainset,
    eval_dataset = valset,
    peft_config=peft_parameters,
    # dataset_text_field=train_dataset["Statement"],
    tokenizer=llama_tokenizer,
    args=train_params,
    #max_seq_length = 2048,
    formatting_func=formatting_prompts_func,
    # data_collator=collator,
)

/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [33]:
fine_tuning.train()

You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.


wandb: Currently logged in as: jbas3235. Use `wandb login --relogin` to force relogin


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
768,1.045200,0.970265
1536,1.042100,0.967326
2304,1.038000,0.964995
3072,1.039200,0.964744
3840,0.895600,0.964325


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=3840, training_loss=1.0367341736331581, metrics={'train_runtime': 38294.4796, 'train_samples_per_second': 0.201, 'train_steps_per_second': 0.1, 'total_flos': 1.5332062395777024e+17, 'train_loss': 1.0367341736331581, 'epoch': 2.0})

In [34]:
fine_tuning.evaluate()


KeyboardInterrupt



In [35]:
fine_tuning.save_model("Cosine_matric_llama2_p1")

In [36]:
fine_tuning.model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (v_

In [37]:
def summarize(text: str):
    inputs = llama_tokenizer(text, return_token_type_ids=False,return_tensors="pt").to("cuda")
#     inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = base_model.generate(**inputs, max_new_tokens=1024, do_sample=True, temperature=0.7)

#         answer_tokens = outputs[:, inputs.input_ids.shape[1] :]
        decoded_output = llama_tokenizer.batch_decode(outputs, skip_special_tokens=True)
        
#         output_text = llama_tokenizer.decode(answer_tokens[0], skip_special_tokens=True).strip()
    return decoded_output

In [38]:
from peft import prepare_model_for_kbit_training

In [39]:
from peft import PeftModel, PeftConfig

In [40]:
output_dir="/kaggle/working/Cosine_matric_mistral_p1"

In [41]:
model_t = PeftModel.from_pretrained(base_model, output_dir)

In [ ]:
# Initialize an empty list to store rows
rows = []

# Iterate over the texts in testset['Meta_Paper']
for i in range(len(testset['Meta_Paper'])):
    text = f"### {DEFAULT_SYSTEM_PROMPT},### Example Abstracts: {testset['Meta_Paper'][i]}  \n ### Meta-Analysis Abstract: "
    processed_output = summarize(text)
    
    # Extract the abstract from processed_output list
    abstract = ""
    for item in processed_output:
        if '### Meta-Analysis Abstract:' in item:
            abstract = item.split('### Meta-Analysis Abstract:')[1].strip()
            break
    
    # Append the "Meta_Paper" value and its processed output as a tuple to the rows list
    rows.append((testset['Meta_Paper'][i], abstract))

# Create a DataFrame from the rows list
df = pd.DataFrame(rows, columns=['Meta_Paper', 'Processed Output'])

# Print or use the DataFrame as needed
print(df)

In [ ]:
for i in range(len(testset['Meta_Paper'])):
    text = f"### {DEFAULT_SYSTEM_PROMPT},### Example Abstracts: {testset['Meta_Paper'][i]}  \n ### Meta-Analysis Abstract: "
    pred = []
    processed_output = summarize(text)
    print(processed_output)
    print("\n\n")

In [ ]:
model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="auto", torch_dtype=torch.bfloat16)
model = model.merge_and_unload()

output_merged_dir = "results/llama2/final_merged_checkpoint"
os.makedirs(output_merged_dir, exist_ok=True)
model.save_pretrained(output_merged_dir, safe_serialization=True)

# save tokenizer for easy inference
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained(output_merged_dir)